In [27]:
import sys
import pickle
from scipy import signal
from scipy import stats
import numpy as np
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import cohen_kappa_score

import math
from collections import OrderedDict


import matplotlib.pyplot as plt

sys.path.append('D:\Diamond\code')
from csp_james_2 import *

sys.path.append('D:\Diamond\code')
from thesis_funcs_19_03 import *

import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler

import csv
import datetime


In [60]:
meth = 'tl_comp_csp_mi' #gold_stand，tl_comp_csp_kld , tl_comp_csp_mi
raw_data_root = 'E:\\Diamond\\bci_iv\\DATA\\2a\\extract_raw\\'
config_root= 'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\configs\\'

feature_root = 'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\CURRENT\\' + meth + '\\'
model_root = feature_root
save_root = model_root + 'eval\\'


#load in cv config grid
hp_names  =[] #all the hyper-parameter names to be validated
with open(config_root +'cv_config.csv', mode = 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    for row in csv_reader:
        hp_names.append((row[0]).strip())

with open(config_root +'_lambda_config.csv', mode = 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    for row in csv_reader:
        hp_names.append((row[0]).strip())
csv_file.close()



num_inits = 5
k_fold = 10

# initialize csp
m = 2# m is Nw in the paper "learning temporal information for brain-copmuter interface, Sakhavi et.al"
n_components = 2 * m  # pick some components
down_sample_step = 20 #Hilbert evelope
# select Ns pairs of csp filters
Ns = 4

CLASSES =[0,1]

C_OVR = [0,1,2,3]

balance_classes = 1

########################################################################################################################
                                            # DEFINE FILTER BANK
########################################################################################################################
#Filter Bank
FB = [[4., 8.], [8., 12.], [12., 16.], [16., 20.], [20., 24.], [24., 28.], [28., 32.], [32., 36.], [36., 40.]]
FB = np.array(FB)

#argumaents for Chebyl II filtering
# Nyquist frequency

# min. attenuation in stop band
gstop = 50
# max. attenuation in passband
gpass= 3 

EEG_PERIOD = [[2.5,6]]
FS = [250]




In [61]:
to_save = 1

In [62]:
save_root  

'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\CURRENT\\tl_comp_csp_mi\\eval\\'

In [63]:
for portion_train in [1,0.7,0.5,0.3,0.1]:
    print ('pt_train', portion_train)

    if to_save == 1:
        filewrite = open(save_root + '4s_' + str(int(portion_train*100))+'_best_config_eval_acc_all_subjects.txt', 'w')
        filewrite.write('')
        filewrite.close()

        filewrite = open(save_root + '4s_' + str(int(portion_train*100))+'_best_config_eval_acc_all_subjects.txt', 'a')
        filewrite.write('subject, ')
        for f in range (0, len(C_OVR)-1):
            filewrite.write('class '+ str(C_OVR[f]+1) + ', ')
        filewrite.write('class ' + str(C_OVR[-1]+1) + ', average'+ ', best_model_init_fold\n')




    for subject in range (1,10):

        filename = 'A0'+str(subject)+'E'
        filename_save = filename
        #print (filename_save)

        eeg_period = EEG_PERIOD[0]
        fs = FS[0]
   
        
        for run_win in range (0,1):
            if run_win == 0:
                file_root_feature =  feature_root + filename_save[:-1] + '\\4s\\' + 'pt_' + str(int(portion_train*100))
                file_root_model = model_root + filename_save[:-1] + '\\4s\\' + 'pt_' + str(int(portion_train*100))
                #file_root_save = save_root + filename_save[:-1] + '\\4s\\' + 'pt_' + str(int(portion_train*100))
                #len_inp = 44
            elif run_win == 1:
                file_root_feature = feature_root + filename_save[:-1] + '\\2s\\' + 'pt_' + str(int(portion_train*100))
                file_root_model = model_root + filename_save[:-1] + '\\2s\\' + 'pt_' + str(int(portion_train*100))
                #file_root_save = save_root + filename_save[:-1] + '\\2s\\' + 'pt_' + str(int(portion_train*100))
                #len_inp = 25
                
            ###################################################################################################################
                            #load best config
            ###################################################################################################################
            #load in best config line
            config_file = open(file_root_model + '\\ANN\\best_config_val.txt', 'r')
            config_log= config_file.readlines()
            config_file.close()
            for i in range (0,len(config_log)):
                line  = config_log[(i + 1) * -1]
                if '_act_fun_' in line: #and  line.split(' ')[0].split('_lambda_')[1] == '0':
                    break

            #extract best config values and make into dictionary
            config = OrderedDict()
            for hp_ind in range(0, len(hp_names)-1):
                config[hp_names[hp_ind]] =  (line.split(hp_names[hp_ind] + '_')[1].split('_'+hp_names[hp_ind+1]+'_')[0])
            config[hp_names[-1]] = line.split(hp_names[-1]+'_')[1].split(' ')[0]
            
            
            ###################################################################################################################
                    #load raw data
            ###################################################################################################################
            #load raw eeg and labels
            EEG_extract_raw = pickle.load(open(raw_data_root
                                                                    + filename_save + '.pickle', 'rb'),encoding='iso-8859-1')
            #Class labels
            LABELS_raw = pickle.load(open(raw_data_root
                                                                + filename_save + "_LABELS.pickle", 'rb'), encoding='iso-8859-1')


            LABELS = LABELS_raw - 1 #already in 1,2,3,4, the labels are. but we nned them to be likw 0,1,2,3
       
            #RUN THIS CODE ONLY ONCE, LABELS0 is the original labels, and LABELS will be changed in one-versus-rest strategy, later
            LABELS0 = LABELS.copy()
            
            
            ########################################################################################################################
                                        #APPLY FILTER BANK
            ########################################################################################################################
            #Store Filter bank filtered raw EEG data, in the shape of num_filter_bank X num_trials X num_chanl X num_samples
            #initiate empty matrix
            EEG_filt_FB_L = np.empty( [len(FB), 
                                     np.shape(EEG_extract_raw)[0], np.shape(EEG_extract_raw)[1],np.shape(EEG_extract_raw)[2]] )

            Nf = fs / 2.
            for fb in range (0, len(FB)):
                passband = FB[fb]
                stopband = FB[fb] + np.array([-2., +2.])

                EEG_filt_FB_L[fb] = filter_signal(EEG_extract_raw, passband, stopband, Nf, gpass, gstop)


            EEG_filt_FB = EEG_filt_FB_L
            
            #trake only the MI 3.5 seconds
            EEG_filt_FB_go = EEG_filt_FB[:,:,:,int(eeg_period[0]*fs):int(eeg_period[1]*fs)]
            LABELS0_go = LABELS0.copy()
            
            
            
            ###########################################################################################################################
    
            pred_indi = []
            ############################################################################################################################
            OUT = 0
            
            best_mod_acc_prod = 0 #initialise best model average class acc
            best_mod_kappa = -2
            best_model = [0,0] #which model performs the best? model id, init = best_model[0], fold = best_model[1]
    
            for fold in range (0, k_fold):
                #print ('fold', fold)

                pred_indi.append([])
                
                for c_ovr in C_OVR:
                    #print (c_ovr)
                    #load in csp filters and mutual informtaion ranked indicies
                    W_B = pickle.load(open( file_root_feature +'\\W_B_fold_' + str(fold) + 
                                               '_c_ovr_' + str(c_ovr) + '_lambda_' + str(float(config['_lambda'])) + 
                                               ".pickle", 'rb'))


                    FB_FILTER_IND = pickle.load(open( file_root_feature +  '\\FB_FILTER_IND_fold_' + str(fold) + 
                                               '_c_ovr_' + str(c_ovr) + '_lambda_' + str(float(config['_lambda'])) + 
                                               ".pickle", 'rb'))

                    #find the selected csp filters indicies
                    FB_FILTER_IND_slt = find_selected_csp_filters(Ns, m, FB_FILTER_IND)

                    #construct selected csp filters, W_B_slt has shape (2*Ns, num_chls), (8,22) for example
                    W_B_slt = W_B[FB_FILTER_IND_slt[:,0], :, FB_FILTER_IND_slt[:,1]]

                    EEG_FB_slt = EEG_filt_FB_go[FB_FILTER_IND_slt[:,0],:]

                    #transform into z space, then take the hilbert envelope of the transformed signal
                    Z_env = calc_z_features(W_B_slt, EEG_FB_slt, Ns, down_sample_step)

                    #concatenate all classes
                    if c_ovr == C_OVR[0]:
                        Z_all_eval = Z_env
                    else:
                        Z_all_eval = np.concatenate((Z_all_eval, Z_env), axis = 0)

                #reshape into ANN input size        
                Z_all_eval = np.transpose(Z_all_eval, [1,0,2])
                X_eval = np.reshape(Z_all_eval, [np.shape(Z_all_eval)[0], 1, np.shape(Z_all_eval)[1], np.shape(Z_all_eval)[2]])
                X_eval = torch.from_numpy(X_eval).float()
                
                #initilize ANN model
                model = Model_current(chn_inp = X_eval.size()[-2], len_inp = X_eval.size()[-1], nf = int(config['nf']), ks = int(config['ks']) , 
                                  stride = int(config['stride']), act_f = config['act_fun'], nfc = int(config['nfc']))
                
                for n_inits in range (0, num_inits):
                    save_path = file_root_model + '\\ANN\\model_config_'+ line.split(' ')[0] + '_n_inits_'+ str(n_inits) +'_fold_' + str(fold) + '.pt'
                    model.load_state_dict(torch.load(save_path))
                    model.eval()

                    #predictoin, sum up the output (probability of being class) predicted at each fold, tehn the class with the max probability if the class prediction
                    out = model(X_eval)
                    OUT = OUT + out

                    #print out class precition at each fold
                    pred = torch.argmax(out, dim = 1).numpy()
                    #print (str(n_inits), np.average(calc_class_acc(pred, LABELS0, C_OVR)))
                    if cohen_kappa_score(LABELS0, pred) > best_mod_kappa:
                        best_model = [n_inits, fold]
                        best_mod_acc_prod = np.average(calc_class_acc(pred, LABELS0, C_OVR))
                        best_mod_kappa = cohen_kappa_score(LABELS0, pred)

                    #pred_indi[fold].append(cohen_kappa_score(LABELS0, pred))
                    
                    
            #final prediciotn using all trained ANNs   
            PRED = torch.argmax(OUT, dim = 1).numpy()
            acc_c = calc_class_acc(PRED, LABELS0, C_OVR)
            print(filename, acc_c, np.average(acc_c))
            kappa = cohen_kappa_score(LABELS0, PRED)
            
            if to_save == 1:
                filewrite.write(str(subject) + ', ')

                for a in acc_c:
                    filewrite.write(str(round(a*100,2)) + ', ')
                filewrite.write(str(round(np.average(acc_c)*100, 2)) +' '+'('+str(round(kappa, 3))+')' +', ' + str(best_model[0])+'_'+str(best_model[1]) + '_'+ str(best_mod_acc_prod) + '(' + str(best_mod_kappa) + ')'  +'\n')
                
                
    if to_save == 1:
        filewrite.close()

pt_train 1
A01E [0.97183099 0.85714286 0.8115942  0.85915493] 0.8749307438835914
A02E [0.36619718 0.57746479 0.95652174 0.72222222] 0.6556014832959107
A03E [0.68656716 1.         0.66176471 0.94117647] 0.8223770851624232
A04E [0.45762712 0.87719298 0.72881356 0.66037736] 0.681002754728202
A05E [0.87142857 0.93846154 0.11111111 0.33333333] 0.5635836385836386
A06E [0.69811321 0.50909091 0.25925926 0.58490566] 0.5128422590686741
A07E [0.85915493 0.95652174 0.81690141 0.93939394] 0.8929930041381358
A08E [0.96969697 0.92647059 0.95652174 0.63235294] 0.8712605595597923
A09E [0.93846154 0.76923077 0.8115942  0.76923077] 0.822129319955407
pt_train 0.7
A01E [0.92957746 0.85714286 0.85507246 0.67605634] 0.8294622809319686
A02E [0.36619718 0.3943662  0.88405797 0.625     ] 0.5674053378240458
A03E [0.65671642 1.         0.63235294 0.91176471] 0.8002085162423178
A04E [0.40677966 0.78947368 0.6779661  0.79245283] 0.6666680692777676
A05E [0.5        0.98461538 0.05555556 0.31884058] 0.464752879970271

In [51]:
best_mod_kappa = -2
str(round(np.average(acc_c)*100, 2)) +' '+'('+str(round(kappa, 3))+')' +', ' + str(best_model[0])+'_'+str(best_model[1]) + '_'+ str(best_mod_acc_prod) + '(' + str(best_mod_kappa) + ')'

'83.33 (0.777), 3_1_0.543014730897462(-2)'

2

In [35]:
best_mod_acc_prod = 0
best_model = [0,0]
str(round(np.average(acc_c)*100, 2)) +' '+'('+str(round(kappa, 3))+')' +', ' + str(best_model[0])+'_'+str(best_model[1])


'88.24 (0.843), 0_0'

In [64]:
cl

NameError: name 'cl' is not defined

In [33]:
np.prod(calc_class_acc(pred, LABELS0, C_OVR)), np.average(calc_class_acc(pred, LABELS0, C_OVR))  

(0.27137472723665934, 0.7477779721809116)

In [34]:
calc_class_acc(pred, LABELS0, C_OVR)

array([0.97183099, 0.54285714, 0.91304348, 0.56338028])

In [24]:
config['_lambda']

'0'

In [16]:
file_root_model

'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\CURRENT\\gold_stand\\A01\\4s\\pt_100'

In [14]:
np.shape(EEG_filt_FB_go), np.shape(LABELS0_go)

((9, 281, 22, 875), (281,))

In [15]:
file_root_feature

'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\CURRENT\\gold_stand\\A01\\4s\\pt_100'